In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
from webdriver_manager.chrome import ChromeDriverManager
from splinter import Browser
import pandas as pd

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
db = client.mars_db
collection = db.articles

In [4]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Driver [C:\Users\Kelly\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


In [5]:
# URL of page to be scraped
mars_url = 'https://mars.nasa.gov/news/'

browser.visit(mars_url)

html = browser.html

# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(html, 'html.parser')

In [6]:
# get the first <li> item under <ul> list of headlines to get latest news title and paragraph text
first_li_item = soup.find('li', class_='slide')

# save news title under the <div> tag with a class of 'content_title'
news_title = first_li_item.find('div', class_='content_title').text

# save paragraph text under the <div> tag with a class of 'article_teaser_body'
news_p = first_li_item.find('div', class_='article_teaser_body').text

print(news_title)
print(news_p)

Testing Proves Its Worth With Successful Mars Parachute Deployment
The giant canopy that helped land Perseverance on Mars was tested here on Earth at NASA’s Wallops Flight Facility in Virginia.


In [7]:
# def featured_image(browser):
#     # Visit URL
#     url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
#     browser.visit(url)

#     # Find and click the full image button
#     full_image_elem = browser.find_by_tag('button')[1]
#     full_image_elem.click()

#     # Parse the resulting html with soup
#     html = browser.html
#     img_soup = soup(html, 'html.parser')

#     # Add try/except for error handling
#     try:
#         # find the relative image url
#         img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')

#     except AttributeError:
#         return None

#     # Use the base url to create an absolute url
#     img_url = f'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/{img_url_rel}'

#     return img_url

In [8]:
# URL of page to be scraped
url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'

browser.visit(url)

# Find and click the full image button
browser.click_link_by_partial_text('FULL IMAGE')

browser.is_element_present_by_text('more info', wait_time=1)

html = browser.html

# Create BeautifulSoup object; parse with 'html.parser'
img_soup = BeautifulSoup(html, 'html.parser')

C:\Users\Kelly\anaconda3\lib\site-packages\splinter\driver\webdriver\__init__.py:490: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  warnings.warn(


In [9]:
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
print(img_url_rel)

image/featured/mars1.jpg


In [10]:
img_url = url + img_url_rel
img_url

'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.htmlimage/featured/mars1.jpg'

In [11]:
# URL of page to be scraped
fact_url = 'https://space-facts.com/mars/'

table = pd.read_html(fact_url)
table

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [12]:
facts_df = table[0]
facts_df.columns =['Description', 'Value']
facts_df

,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [13]:
facts_df.to_html('marsfacts.html', index=False)

In [14]:
hem_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'


In [15]:
browser.visit(hem_url)

In [16]:
# create HTML object
html = browser.html
# print(html)

In [17]:
# parse HTML with BeautifulSoup
hem_soup = BeautifulSoup(html, 'html.parser')

In [20]:
# get all the parent div tags for each hemisphere
hem_divs = hem_soup.find_all('div', class_="item")

# create empty list to store the python dictionary
hemisphere_image_urls = []

# loop through each div item to get hemisphere data
for hem in range(len(hem_divs)):

    # use splinter to click on each hemisphere link to get image data
    hem_link = browser.find_by_css("a.product-item h3")
    hem_link[hem].click()
    
    # create beautiful soup object with the image detail page html
    img_detail_html = browser.html
    img_detail_soup = BeautifulSoup(img_detail_html, 'html.parser')
    
    # create the base url for link to full size image
    base_url = 'https://astrogeology.usgs.gov'
    
    # get gull size image url and save to a variable
    hem_url = img_detail_soup.find('img', class_="wide-image")['src']
    
    # get whole featured image url
    feat_img_url = base_url + hem_url

    # get image title using the title class and save to variable
    feat_img_title = browser.find_by_css('.title').text
    
    # append values to a dictionary
    hemisphere_image_urls.append({"title": feat_img_title,
                        "img_url": feat_img_url})
    
    # go back to main page
    browser.back()

hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]